# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Installing citipy via jupyter notebook
# to determine city based on latitude and longitude
# ! pip install citipy

In [2]:
# Importing packages and API key
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from citipy import citipy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Remember to enter in the cell twice for this to work
from weatherweatherlang import owm_key

# Creating variables for range for latitude and longitude
lat_range = (-90, 90)
lng_range = (-180, 180)

# Creating lists for holding lat_lngs, cities, and country codes
lat_lngs = []
cities = []
countries = []

## Generate Cities List

In [3]:
# When running, need to reset everything so that we're 
# not generating too many API calls

# Creating a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identifing nearest city for each lat, lng combination
# (Using functions and variables already defined in citipy)
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
# Making sure that duplicate cities are not added by making
# appends conditional
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Printing the city count to confirm sufficient 
# count for JSON requests
len(cities)

600

In [4]:
# Making an initial dataframe for these pairings
citiweather_df = pd.DataFrame({'City':cities, 'Country Code':countries})

# Checking if city-country code pairings are accurate
citiweather_df.head(3)
citiweather_df.tail(3)

# Adding additional columns to dataframe to add from later OWM requests
citiweather_df['Lat'] = ""
citiweather_df['Lng'] = ""
citiweather_df['Date'] = ""
citiweather_df['Temp (F)'] = ""
citiweather_df['Humidity'] = ""
citiweather_df['Cloudiness'] = ""
citiweather_df['Wind Speed'] = ""
citiweather_df.head(3)

,City,Country Code
0,chara,ru
1,chokurdakh,ru
2,upernavik,gl


,City,Country Code
597,aflu,dz
598,tabulbah,tn
599,nichinan,jp


,City,Country Code,Lat,Lng,Date,Temp (F),Humidity,Cloudiness,Wind Speed
0,chara,ru,,,,,,,
1,chokurdakh,ru,,,,,,,
2,upernavik,gl,,,,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Building the JSON request loop by searching with city
# and country code pairs, and a print log of each city as
# the JSON is being processed

json_counter = 0

f'Initiating data retrieval from OpenWeatherMap API...'
f'----------------------------------------------------'

for index, row in citiweather_df.iterrows():
    # OWM endpoint url with imperial units to get Fahrenheit
    # temperature and miles per hour wind speed
    # (Refer back to https://openweathermap.org/current for parameters)
    owm_url = ('http://api.openweathermap.org/data/2.5/weather?q={0},{1}&units=imperial&APPID={2}').format(row['City'],row['Country Code'],owm_key)
    owm_response = requests.get(owm_url).json()
    f"Processing request for city {json_counter + 1}: {citiweather_df.loc[index]['City']}"
    owm_url
    json_counter += 1
    f'----------------------------------------------------'
    
    try:
    # Retrieving specific datapoints from each JSON request and 
    # putting them into the dataframe 
        citiweather_df.loc[index, 'Lat'] = owm_response['coord']['lat'] # latitude
        citiweather_df.loc[index, 'Lng'] = owm_response['coord']['lng'] # longitude
        citiweather_df.loc[index, 'Date'] = owm_response['dt'] # date results retrieved 
        citiweather_df.loc[index, 'Temp (F)'] = owm_response['main']['temp'] # Farhenheit temperature
        citiweather_df.loc[index, 'Humidity'] = owm_response['main']['humidity'] # humidity
        citiweather_df.loc[index, 'Cloudiness'] = owm_response['clouds']['all'] # cloudliness
        citiweather_df.loc[index, 'Wind Speed'] = owm_response['wind']['speed'] # wind speed  
    
    except KeyError:
        f'City not found. Proceeding to next city on list...'
    
    # Using a modulo and time here to pause requests as necessary
    if json_counter % 60 == 0:
        f'----------------------------------------------------'
        f'Pausing API requests for 30 seconds.'
        f"(Don't want to overload my free account here.)"
        f'----------------------------------------------------' 
        time.sleep(15)

f'----------------------------------------------------'
f'Now concluding data retrieval from OpenWeatherMap API.'
f'                  Have a nice day!'
f'----------------------------------------------------'

In [8]:
# Displaying initial results
citiweather_df.head(3)

# Converting date value to datetime
citiweather_df['Date'] = pd.to_datetime(citiweather_df['Date'], unit='s')
citiweather_df.head(3)

# Finding the number of rows in dataframe
len(citiweather_df.index)

,City,Country Code,Lat,Lng,Date,Temp (F),Humidity,Cloudiness,Wind Speed
0,chara,ru,56.9,,,,,,
1,chokurdakh,ru,70.62,,,,,,
2,upernavik,gl,72.79,,,,,,


,City,Country Code,Lat,Lng,Date,Temp (F),Humidity,Cloudiness,Wind Speed
0,chara,ru,56.9,,NaT,,,,
1,chokurdakh,ru,70.62,,NaT,,,,
2,upernavik,gl,72.79,,NaT,,,,


600

In [ ]:
# Exporting results to csv file


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot